In [1]:
import pandas as pd
import requests, json
import numpy as np
import time

# Connection Lost

In [2]:
# Make a header for the request
# Header has been copied from Mozila - Network - Requests - POST - json
headers = '''
Host: auto.ru
User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:88.0) Gecko/20100101 Firefox/88.0
Accept: */*
Accept-Language: ru-RU,ru;q=0.8,en-US;q=0.5,en;q=0.3
Accept-Encoding: gzip, deflate, br
Referer: https://auto.ru/cars/bmw/all/?page=3
x-client-app-version: bce8a8244b5
x-page-request-id: 328918ce0247597eaf5327bfb58fbbf0
x-client-date: 1621052424675
x-csrf-token: 46e8e9805b1aaeb0e7e8ed06e9e61db609f825f0374f514d
x-requested-with: fetch
content-type: application/json
Origin: https://auto.ru
Content-Length: 78
Connection: keep-alive
Cookie: autoru_sid=a%3Ag609e770a22hv5h9nuukn7hehqvu3kep.f63513a378b5f5c6208d35b64697caa2%7C1620997898875.604800.wyDgKeg1mlEd3UgRUP-bJA.67jb8JxaCRrLGKWbo9PTOIBxTdPMTL7-ErqZR1uTZ90; autoruuid=g609e770a22hv5h9nuukn7hehqvu3kep.f63513a378b5f5c6208d35b64697caa2; autoru_gdpr=1; suid=c18f94687848ae634a3e33b24dec6d0a.4fc40ee98cea6dd87fe788cb17ac994d; yuidlt=1; yandexuid=3854193761620997913; counter_ga_all7=2; crookie=An+ejc3GId86JbaHno+3yDpWbPhSpPl+/G5iLmug8RDpM4VJDMjLzHii5iDi/5MZT+TrlHvnUdD/Khj0WeuIogIWnpE=; cmtchd=MTYyMDk5NzkyNzY0NA==; _ym_uid=162099793885069073; _ym_d=1621052333; _ym_isad=2; cycada=1+Z/guXKzEVROtphAKfMJ/7TQUH18nTGnu0U0PuR/rk=; _ga=GA1.2.1678141264.1620998016; _gid=GA1.2.1513849119.1620998016; parts_ur=; _csrf_token=46e8e9805b1aaeb0e7e8ed06e9e61db609f825f0374f514d; from_lifetime=1621052333763; from=direct; X-Vertis-DC=vla; gdpr=0
Pragma: no-cache
Cache-Control: no-cache
'''.strip().split("\n")

# Convert request to dictionary
dict_header = {}
for header in headers:
    key, value = header.split(': ')
    dict_header[key] = value
print(dict_header)

{'Host': 'auto.ru', 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:88.0) Gecko/20100101 Firefox/88.0', 'Accept': '*/*', 'Accept-Language': 'ru-RU,ru;q=0.8,en-US;q=0.5,en;q=0.3', 'Accept-Encoding': 'gzip, deflate, br', 'Referer': 'https://auto.ru/cars/bmw/all/?page=3', 'x-client-app-version': 'bce8a8244b5', 'x-page-request-id': '328918ce0247597eaf5327bfb58fbbf0', 'x-client-date': '1621052424675', 'x-csrf-token': '46e8e9805b1aaeb0e7e8ed06e9e61db609f825f0374f514d', 'x-requested-with': 'fetch', 'content-type': 'application/json', 'Origin': 'https://auto.ru', 'Content-Length': '78', 'Connection': 'keep-alive', 'Cookie': 'autoru_sid=a%3Ag609e770a22hv5h9nuukn7hehqvu3kep.f63513a378b5f5c6208d35b64697caa2%7C1620997898875.604800.wyDgKeg1mlEd3UgRUP-bJA.67jb8JxaCRrLGKWbo9PTOIBxTdPMTL7-ErqZR1uTZ90; autoruuid=g609e770a22hv5h9nuukn7hehqvu3kep.f63513a378b5f5c6208d35b64697caa2; autoru_gdpr=1; suid=c18f94687848ae634a3e33b24dec6d0a.4fc40ee98cea6dd87fe788cb17ac994d; yuidlt=1; yandexuid=3854193

In [3]:
# Address where is a json file with all AD
url = "https://auto.ru/-/ajax/desktop/listing/"

In [4]:
# Specify brands to be scraped
brand_list = ['BMW',
              'VOLKSWAGEN',
              'NISSAN',
              'MERCEDES',
              'TOYOTA',
              'AUDI',
              'MITSUBISHI',
              'SKODA',
              'VOLVO',
              'HONDA',
              'INFINITI',
              'LEXUS',
              'FORD',
              'KIA',
              'HYUNDAI',
              'MAZDA',
              'RENO',
              'Peugeot'
]

In [5]:
# Make sraper
def parser_v2(model, page_range):
    #Here we save all offers from each page
    offers = []
    # Break our search by years 
    # (Auto.ru displayed max 100 pages. so Use years to gather more AD)
    years = [1900, 2000, 2010, 2020, 2021]
    end_year = 2000
    for start_year in years:
        try:
            for x in range(1, page_range):
                # For safety add freeze time (avoid to be banned)
                time.sleep(.2)
                param = {
                    "color": [
                        "FFD600", "660099", "DEA522", "007F00", "040001",
                        "C49648", "CACECB", "EE1D19", "0000CC", "22A0F8",
                        "FFC0CB", "4A2197", "FF8649", "200204", "97948F",
                        "FAFBFB"
                    ],
                    "year_from":
                    start_year,
                    "year_to":
                    end_year,
                    "catalog_filter": [{
                        "mark": model
                    }],
                    "damage_group":
                    "ANY",
                    'sort':
                    "fresh_relevance_1-desc",
                    "top_days":
                    "900",
                    "customs_state_group":
                    "DOESNT_MATTER",
                    "section":
                    "all",
                    "category":
                    "cars",
                    "page":
                    x
                }

                response = requests.post(url, json=param,
                                         headers=dict_header)  # steal json
                print(response.status_code)  # Print request status
                data = response.json()  # extract what we stolen :)
                # Take only cars offers. Avoid taking trash
                offers.extend(data['offers'])
                print(f'Current_page: {x}')  # current page No.
                print(start_year, end_year)
        except:
            print('Fail')
        if end_year >= 2020:
            end_year = 2021
        else:
            end_year += 10
    with open(f'data_{model}.json', 'w') as f:
        json.dump(offers, f)
    print(f'File "data_{model}" is ready')

In [ ]:
 # Run it
for i in brand_list:
    parser_v2(i,100)

In [7]:
# Create cols name(same as in test data)
cols = [
    'bodyType', 'brand', 'car_url', 'color', 'complectation_dict',
    'description', 'engineDisplacement', 'enginePower', 'equipment_dict',
    'fuelType', 'image', 'mileage', 'modelDate', 'model_info', 'model_name',
    'name', 'numberOfDoors', 'parsing_unixtime', 'priceCurrency',
    'productionDate', 'sell_id', 'super_gen', 'vehicleConfiguration',
    'vehicleTransmission', 'vendor', 'Владельцы', 'Владение', 'ПТС', 'Привод',
    'Руль', 'Состояние', 'Таможня', 'price'
]  

In [8]:
def get_df(parsed_json):
    '''Function is called to extract data from Json and convert to Pandas data frame'''
    df = pd.DataFrame(data=None, columns=cols, index=range(len(parsed_json)))
    NaN = np.nan
    for i, vehicle in enumerate(parsed_json):
        try:
            df.loc[i]['color'] = vehicle['color_hex']
        except:
            df.loc[i]['color'] = NaN
        try:
            df.loc[i].brand = vehicle['vehicle_info']['mark_info']['name']
        except:
            df.loc[i].brand = NaN
        try:
            df.loc[i].complectation_dict = vehicle['vehicle_info'][
                'complectation']
        except:
            df.loc[i].complectation_dict = NaN
        try:
            df.loc[i].engineDisplacement = vehicle['vehicle_info'][
                'tech_param']['displacement']
        except:
            df.loc[i].engineDisplacement = NaN
        try:
            df.loc[i].enginePower = vehicle['vehicle_info']['tech_param'][
                'power']
        except:
            df.loc[i].enginePower = NaN
        try:
            df.loc[i].equipment_dict = vehicle['vehicle_info']['equipment']
        except:
            df.loc[i].equipment_dict = NaN
        try:
            df.loc[i].fuelType = vehicle['lk_summary'].split(',')[-1]
        except:
            df.loc[i].fuelType = NaN
        try:
            df.loc[i].image = vehicle['state']['image_urls'][0]['sizes'][
                '1200x900n']
        except:
            df.loc[i].image = NaN
        try:
            df.loc[i].mileage = vehicle['state']['mileage']
        except:
            df.loc[i].mileage = NaN
        try:
            df.loc[i].modelDate = vehicle['vehicle_info']['super_gen'][
                'year_from']
        except:
            df.loc[i].modelDate = NaN
        try:
            df.loc[i].model_info = vehicle['vehicle_info']['model_info']
        except:
            df.loc[i].model_info = NaN
        try:
            df.loc[i].model_name = vehicle['vehicle_info']['model_info'][
                'code']
        except:
            df.loc[i].model_name = NaN
        try:
            df.loc[i]['name'] = vehicle['vehicle_info']['tech_param'][
                'human_name']
#             df.loc[i]['name'] = vehicle['lk_summary']  # !!!!!!!!!!!!!!!!!!!
        except:
            df.loc[i]['name'] = NaN
        try:
            df.loc[i].numberOfDoors = vehicle['vehicle_info']['configuration'][
                'doors_count']
        except:
            df.loc[i].numberOfDoors = NaN
        try:
            df.loc[i].parsing_unixtime = 'today'
        except:
            df.loc[i].parsing_unixtime = NaN
        try:
            df.loc[i].productionDate = vehicle['documents']['year']
        except:
            df.loc[i].productionDate = NaN
        try:
            df.loc[i].sell_id = vehicle['id']
        except:
            df.loc[i].sell_id = NaN
        try:
            df.loc[i].super_gen = vehicle['vehicle_info']['tech_param']  #!!!
        except:
            df.loc[i].super_gen = NaN
        try:
            df.loc[i].vehicleConfiguration = 'useless_col'
        except:
            df.loc[i].vehicleConfiguration = NaN
        try:
            df.loc[i].vehicleTransmission = vehicle['vehicle_info'][
                'tech_param']['transmission']
        except:
            df.loc[i].vehicleTransmission = NaN
        try:
            df.loc[i].vendor = vehicle['vehicle_info']['vendor']
        except:
            df.loc[i].vendor = NaN
        try:
            df.loc[i].Владение = vehicle['documents']['purchase_date']  #!!!
        except:
            df.loc[i].Владение = NaN
        try:
            df.loc[i].Привод = vehicle['lk_summary'].split(',')[2]
        except:
            df.loc[i].Привод = NaN
        try:
            df.loc[i].Состояние = vehicle['state']['state_not_beaten']
        except:
            df.loc[i].Состояние = NaN
        try:
            df.loc[i].Таможня = vehicle['documents']['custom_cleared']
        except:
            df.loc[i].Таможня = NaN


#         try:
#             df.loc[i].price_segment = vehicle['vehicle_info']['super_gen']['price_segment']
#         except:
#             df.loc[i].price_segmen = NaN
        try:
            df.loc[i].bodyType = vehicle['vehicle_info']['configuration'][
                'human_name']
        except:
            df.loc[i].bodyType = NaN
        try:
            df.loc[i].description = vehicle['description']
        except:
            df.loc[i].description = NaN
        try:
            df.loc[i].Владельцы = vehicle['documents']['owners_number']
        except:
            df.loc[i].Владельцы = NaN
        try:
            df.loc[i].price = vehicle['price_info']['price']
        except:
            df.loc[i].price = NaN
        try:
            df.loc[i].priceCurrency = vehicle['price_info']['currency']
        except:
            df.loc[i].priceCurrency = NaN
        try:
            df.loc[i].ПТС = vehicle['documents']['pts']
        except:
            df.loc[i].ПТС = NaN
        try:
            df.loc[i].Руль = vehicle['vehicle_info']['steering_wheel']
        except:
            df.loc[i].Руль = NaN
        try:
            df.loc[i].price_USD = vehicle['price_info']['USD']
        except:
            df.loc[i].price_USD = NaN

    return df

In [10]:
def get_train(car_name):
    '''Func to open each sraped Json it send it to get_df func'''
    df = pd.DataFrame(data=None)
    with open (f'data_{car_name}.json', 'r', encoding='utf8') as f:
        data = json.load(f)
    df = get_df(data)
    return df

In [11]:
# Make each car brands DF
AUDI = get_train('AUDI')
BMW = get_train('BMW')
HONDA = get_train('HONDA')
INFINITI = get_train('INFINITI')
LEXUS = get_train('LEXUS')
MERCEDES = get_train('MERCEDES')
MITSUBISHI = get_train('MITSUBISHI')
NISSAN = get_train('NISSAN')
SKODA = get_train('SKODA')
TOYOTA = get_train('TOYOTA')
VOLKSWAGEN = get_train('VOLKSWAGEN')
FORD = get_train('FORD')
HYUNDAI = get_train('HYUNDAI')
KIA = get_train('KIA')
VOLVO = get_train('VOLVO')
MAZDA = get_train('MAZDA')
# RENO = get_train('RENO')
# Peugeot = get_train('Peugeot')

FileNotFoundError: [Errno 2] No such file or directory: 'data_AUDI.json'

In [ ]:
# Combine them
df_list = [BMW,
           VOLKSWAGEN,NISSAN,
           MERCEDES,TOYOTA,AUDI,
           MITSUBISHI,SKODA,VOLVO,
           HONDA,INFINITI,LEXUS,
           FORD,KIA,HYUNDAI,MAZDA,
#            RENO,Peugeot
          ]
df = pd.concat(df_list, sort=False)

In [ ]:
# Save to csv
df.to_csv('train_parsed_24_05_21.csv', index = False)

In [ ]:
# Read it
df_train = pd.read_csv('train_parsed_24_05_21.csv')

In [ ]:
# And drop duplicates
df_train.duplicated(keep='first').sum()

In [ ]:
df_train = df_train.drop_duplicates()

In [ ]:
df_train.duplicated(keep='first').sum()